In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices()
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

2022-05-26 13:38:47.335763: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-05-26 13:38:49.048488: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-26 13:38:49.049342: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-26 13:38:49.094801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 13:38:49.095068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-05-26 13:38:49.095093: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-26 13:38:49.096858: I tensorflow/stream_executor/platform/def

In [2]:
import numpy as np
import math
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LayerNormalization
from collections import deque
import gym
import os
from tensorflow.keras.optimizers import Adam

In [3]:
env = gym.make("BipedalWalker-v3")
input_size = env.observation_space.shape[0]
output_size = env.action_space.shape[0]
input_size, output_size

(24, 4)

In [15]:
env = gym.wrappers.RecordVideo(env, "videos", step_trigger=lambda t: t%10_000==0)

In [5]:
tau = 0.001             #soft target update
gamma = 0.99             #discount factor


actor_lr = 0.0001       
critic_lr = 0.001

memory_size = int(1e6)
minibatch_size = 128

num_episodes = 2_000   #max episodes for training
num_steps=700          #max number of steps in an episode

noise_std = 0.2        #stddev of noise added for exploration
start_after = 1000     #pure exploration before 1000 steps
update_after = 1000    #start learning after 1000 steps

l2_decay=0.001         #weight decay for critic network

In [6]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# rm -rf ./logs/train_PPO/
train_log_dir = "logs/train_DDPG/" + current_time
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
%load_ext tensorboard
%tensorboard --logdir {train_log_dir}

2022-05-26 13:38:49.258297: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-26 13:38:49.259134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 13:38:49.259489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-05-26 13:38:49.259536: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

In [7]:
class ReplayMemory:
    def __init__(self, input_size, output_size, size):
        self.states = np.zeros(shape = (size, input_size), dtype=np.float32)
        self.next_states = np.zeros(shape = (size, input_size), dtype=np.float32)
        self.actions = np.zeros(shape = (size, output_size), dtype=np.float32)
        self.rewards = np.zeros(shape = size, dtype=np.float32)
        self.dones = np.zeros(shape = size, dtype=np.float32)
        self.pointer=0
        self.size=0
        self.max_size=size
    
    def remember(self, state, action, reward, next_state, done):
        self.states[self.pointer] = state
        self.next_states[self.pointer] = next_state
        self.actions[self.pointer] = action
        self.rewards[self.pointer] =  reward
        self.dones[self.pointer] = done
        self.pointer = (self.pointer + 1)%self.max_size
        self.size = min(self.size+1, self.max_size)
        
    def sample(self, minibatch_size = 32):
        idxs = np.random.randint(0, self.size, size=minibatch_size)
        
        return self.states[idxs],\
               self.actions[idxs],\
               self.rewards[idxs],\
               self.next_states[idxs],\
               self.dones[idxs]
                
    

In [8]:
# Taken from OpenAI baselines
# https://github.com/openai/baselines/blob/master/baselines/ddpg/noise.py

class AdaptiveParamNoiseSpec(object):
    def __init__(self, initial_stddev=0.1, desired_action_stddev=0.1, adoption_coefficient=1.01):
        self.initial_stddev = initial_stddev
        self.desired_action_stddev = desired_action_stddev
        self.adoption_coefficient = adoption_coefficient
        self.current_stddev = initial_stddev
       

    def adapt(self, distance):
        if distance > self.desired_action_stddev:
            # Decrease stddev.
            self.current_stddev /= self.adoption_coefficient
        else:
            # Increase stddev.
            self.current_stddev *= self.adoption_coefficient
        
    
    def get_std(self):
        return self.current_stddev
    
def distance_metric(action1, action2):
    diff = action1 - action2
    mean_diff = np.mean(np.square(diff), axis=0)
    dist = np.sqrt(np.mean(mean_diff))
    return dist

In [9]:
class Agent:
    def __init__(self):
        self.param_noise = AdaptiveParamNoiseSpec()
        
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        self.noisy_actor = self.build_actor()
        
        self.target_actor = self.build_actor()
        self.target_critic = self.build_critic()
        
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())
        
        self.actor_optimizer = Adam(lr=actor_lr)
        self.critic_optimizer =  Adam(lr=critic_lr)
        
        self.memory = ReplayMemory(input_size, output_size, memory_size)
        
        
    def build_actor(self):
        init = tf.random_uniform_initializer(minval=-3e-3, maxval=3e-3)
        inputs = Input(shape = input_size)
        x = Dense(400)(inputs)
        x = LayerNormalization()(x)
        x = tf.nn.relu(x)
        x = Dense(300)(x)
        x = LayerNormalization()(x)
        x = tf.nn.relu(x)
        x= Dense(100)(x)
        x = LayerNormalization()(x)
        x = tf.nn.relu(x)
        x = Dense(output_size, kernel_initializer=init)(x)
        x = LayerNormalization()(x)
        x = tf.nn.tanh(x)
        model = Model(inputs = inputs, outputs = x)
        return model
    
    def build_critic(self):
        init = tf.random_uniform_initializer(minval=-3e-3, maxval=3e-3)
        regularizer = tf.keras.regularizers.L2(l2=l2_decay)
        input1 = Input(shape = input_size)
        input2 = Input(shape = output_size)
        x = Dense(400, activation="relu", kernel_regularizer=regularizer)(input1)
        x = Dense(300, activation="relu", kernel_regularizer=regularizer)(tf.concat([x, input2], axis =1))
        x = Dense(100, activation="relu", kernel_regularizer=regularizer)(x)
        x = Dense(1, activation = 'linear', kernel_initializer=init, kernel_regularizer=regularizer)(x)
        model = Model(inputs = [input1, input2], outputs = x)
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    @tf.function 
    def act(self, state, test = False):
        if test:
            return self.actor(state)
        else:
            return self.noisy_actor(state)
    
    def perturb_actor_params(self):
        weights = []
        for weight in self.actor.get_weights():
            noise = np.random.normal(scale = self.param_noise.current_stddev, size = weight.shape)
            weights.append(weight + noise) 
        self.noisy_actor.set_weights(weights)
    
    def adapt_noise_parameter(self):
        if agent.memory.size<minibatch_size:
            return
        states, noisy_actions, _, _, _ = self.memory.sample(minibatch_size)
        og_actions = self.actor(states)
        distance = distance_metric(og_actions, noisy_actions)
        print(f"Distance: {distance}")
        self.param_noise.adapt(distance)

    @tf.function
    def optimize(self, states, actions, rewards, next_states, dones):
        rewards= tf.expand_dims(rewards, axis=1)
        dones = tf.expand_dims(dones, axis=1)
        next_actions = self.target_actor(next_states)
        next_values = self.target_critic([next_states, next_actions])
        target_q_values = rewards + gamma * (1-dones) * next_values
        with tf.GradientTape() as tape:        
            pred_q_values = self.critic([states, actions])
            critic_loss = tf.reduce_mean((target_q_values - pred_q_values)**2)
        critic_grads = tape.gradient(critic_loss, self.critic.trainable_weights)
        self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_weights))
        
        with tf.GradientTape() as tape:
            actions_pred = self.actor(states)
            actor_loss = -self.critic([states, actions_pred])
            actor_loss = tf.reduce_mean(actor_loss)
        actor_grads = tape.gradient(actor_loss, self.actor.trainable_weights)
        self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_weights))
    
    def learn(self):
        if agent.memory.size<minibatch_size:
            return 
        states, actions, rewards, next_states, dones = self.memory.sample(minibatch_size)
        self.optimize(states, actions, rewards, next_states, dones) 
        self.update_target_weights()
        self.perturb_actor_params()
    
    def update_target_weights(self):
        target_actor_weights = self.target_actor.get_weights()
        weights=[]
        for i, weight in enumerate(self.actor.get_weights()):
            weights.append(weight * tau + target_actor_weights[i] * (1-tau))
        self.target_actor.set_weights(weights)
        
                           
        target_critic_weights = self.target_critic.get_weights()
        weights=[]
        for i, weight in enumerate(self.critic.get_weights()):
            weights.append(weight * tau + target_critic_weights[i] * (1-tau))
        self.target_critic.set_weights(weights)
                           


    def load(self, name):
        self.actor.load_weights(name + "actor.hdf5")
        self.critic.load_weights(name + "critic.hdf5")
        self.target_actor.load_weights(name + "actor.hdf5")
        self.target_critic.load_weights(name + "critic.hdf5")
    
    def save(self, name):
        self.actor.save_weights(name + "actor.hdf5")
        self.critic.save_weights(name + "critic.hdf5")    

In [10]:
agent = Agent()

In [ ]:
#Training 
best_score = env.reward_range[0]
score_history = deque(maxlen=100)
global_step = 0
agent.perturb_actor_params()
for ep in range(0, num_episodes):
    state = env.reset()
    done = False
    score = 0
    agent.adapt_noise_parameter()
    print(f"Current Stddev: {agent.param_noise.current_stddev}") 
    for step in range(num_steps):
        
        action =  agent.act(np.expand_dims(state, axis = 0))
        action = np.squeeze(action)  
        next_state, reward, done, _ =  env.step(action)
        score += reward
        global_step += 1
        agent.memory.remember(state, action, reward, next_state, done)
        state = next_state
        if global_step>update_after:
            agent.learn()
            
        if done:
            break
    with train_summary_writer.as_default():
        tf.summary.scalar("Charts/score", score, global_step)
        tf.summary.scalar("Charts/episode_length", step, global_step)
        tf.summary.scalar("Charts/exploration", noise_std, global_step)
        
    score_history.append(score)
    avg_score = np.mean(score_history)
    print(f"Episode: {ep}, Len: {step}, Score: {score}, Avg Score: {avg_score}, Global_Step: {global_step}")  
    if avg_score > best_score:
        best_score = score
        agent.save("DDPG_") 
    if ep%25==0:
        print(f"Testing")
        for _ in range(2):
            state=env.reset()
            done = False
            test_score=0
            test_step=0
            while not done:
                action = agent.act(np.expand_dims(state, axis =0), test=True)
                next_state, reward, done, _ = env.step(np.squeeze(action))
                test_score+= reward
                test_step+=1
                state=next_state
            print(f"Episode ended with length {test_step} and a score of {test_score}")


In [ ]:
# agent.save("Latest_")

In [11]:
# #Loading pretrained weights

agent.load("Latest_")

In [16]:
#Testing
done = False
state = env.reset()
score=0
step = 0
while not done:
    env.render()
    action = agent.act(np.expand_dims(state, axis=0), test=True)
    next_state, reward, done, _ = env.step(np.squeeze(action))
    score+= reward
    state = next_state
    print(action, reward)
    step += 1
print(f"Episode ended in {step} steps with score of {score}")
env.close()

tf.Tensor([[-0.9997906  -0.99943846  0.99999976 -0.99998635]], shape=(1, 4), dtype=float32) -0.20793367404552915
tf.Tensor([[-0.9967651  -0.99212563  0.99999976 -0.9999986 ]], shape=(1, 4), dtype=float32) -0.08564096299745143
tf.Tensor([[-0.9836913  -0.99455017  0.99999976 -0.9999992 ]], shape=(1, 4), dtype=float32) -0.20920508500002324
tf.Tensor([[-0.645001   -0.9722766   0.9999993  -0.99999976]], shape=(1, 4), dtype=float32) -0.21251986015339575
tf.Tensor([[-0.18393105 -0.8305299   0.99999845 -0.99999976]], shape=(1, 4), dtype=float32) -0.1920588602026292
tf.Tensor([[-0.09420314  0.50117743  0.99999523 -0.99999976]], shape=(1, 4), dtype=float32) -0.17186609061559277
tf.Tensor([[ 0.8768979   0.998758    0.999865   -0.99999976]], shape=(1, 4), dtype=float32) -0.16301054354508598
tf.Tensor([[ 0.1353933   0.97818804  0.99998224 -0.99999976]], shape=(1, 4), dtype=float32) -0.13536124287049095
tf.Tensor([[ 0.12900545  0.97529054  0.9999829  -0.99999976]], shape=(1, 4), dtype=float32) -0.11

tf.Tensor([[ 0.99999976 -0.93586427  0.779829   -0.99999976]], shape=(1, 4), dtype=float32) 0.2655827946787103
tf.Tensor([[ 0.99999976 -0.54303306  0.5205884  -0.99999976]], shape=(1, 4), dtype=float32) 0.10266022767374679
tf.Tensor([[ 0.99999976 -0.99457884  0.22119337 -0.99999976]], shape=(1, 4), dtype=float32) 0.031641163220010586
tf.Tensor([[ 0.99999976 -0.9982711  -0.9999737  -0.9986964 ]], shape=(1, 4), dtype=float32) 0.07907499317327901
tf.Tensor([[-0.99983096  0.99999475 -0.99999976  0.99932146]], shape=(1, 4), dtype=float32) 0.2778012505571095
tf.Tensor([[ 0.99999976  0.98614615  0.964217   -0.99999976]], shape=(1, 4), dtype=float32) 0.07644200167059902
tf.Tensor([[ 0.99999976  0.35603875  0.99298686 -0.99999976]], shape=(1, 4), dtype=float32) -0.024226539323732026
tf.Tensor([[ 0.99992615  0.99999195  0.78886557 -0.99999976]], shape=(1, 4), dtype=float32) -0.08824904495477677
tf.Tensor([[-0.5763604  -0.80387086  0.9999983  -0.99999976]], shape=(1, 4), dtype=float32) 0.08372399

tf.Tensor([[ 0.99704313  0.99846303  0.9994105  -0.99999976]], shape=(1, 4), dtype=float32) 0.17319369298219678
tf.Tensor([[ 0.99999976  0.41088694  0.9692674  -0.99999976]], shape=(1, 4), dtype=float32) 0.11589800719419878
tf.Tensor([[-0.9993215   0.98270017 -0.9999997   0.9999855 ]], shape=(1, 4), dtype=float32) 0.18460756719112398
tf.Tensor([[-0.92205256  0.97691804  0.99998987 -0.99999976]], shape=(1, 4), dtype=float32) 0.1210800234079361
tf.Tensor([[-0.9979745   0.99999976 -0.9999985   0.8156771 ]], shape=(1, 4), dtype=float32) 0.17704354818662243
tf.Tensor([[-0.7709161   0.576358   -0.99999976  0.9999909 ]], shape=(1, 4), dtype=float32) 0.23292975662151735
tf.Tensor([[ 0.99999976 -0.9999871  -0.99974376 -0.9990437 ]], shape=(1, 4), dtype=float32) 0.1909281952381134
tf.Tensor([[ 0.99999976 -0.9991811  -0.9999737  -0.99861753]], shape=(1, 4), dtype=float32) 0.1908556233048439
tf.Tensor([[ 0.99999976 -0.9955784  -0.9999985  -0.9603995 ]], shape=(1, 4), dtype=float32) 0.1921188895305

tf.Tensor([[ 0.99999976  0.99989283 -0.9999787  -0.99999577]], shape=(1, 4), dtype=float32) 0.18628567941983068
tf.Tensor([[-0.99999547  0.9999964  -0.99999803  0.99908364]], shape=(1, 4), dtype=float32) 0.2281321517825127
tf.Tensor([[-0.99999565  0.46655992 -0.9999946   0.99999803]], shape=(1, 4), dtype=float32) 0.3037293442090305
tf.Tensor([[ 0.99999976 -0.95156574 -0.99999976  0.5023994 ]], shape=(1, 4), dtype=float32) 0.2878826876680104
tf.Tensor([[ 0.99999976  0.7759117  -0.99999976  0.784022  ]], shape=(1, 4), dtype=float32) 0.3052622822523117
tf.Tensor([[-0.9903436  -0.99849725 -0.99998075  0.9999968 ]], shape=(1, 4), dtype=float32) 0.3124249970565287
tf.Tensor([[-0.99994665 -0.9971771  -0.99968153  0.99999535]], shape=(1, 4), dtype=float32) 0.26027035381396135
tf.Tensor([[ 0.9991197   0.98112595 -0.99999976  0.99805295]], shape=(1, 4), dtype=float32) 0.10003960228960193
tf.Tensor([[-0.74962884 -0.9997488   0.9999997  -0.99999875]], shape=(1, 4), dtype=float32) 0.125058539842561

tf.Tensor([[-0.97816336  0.9992703   0.9999774  -0.99999976]], shape=(1, 4), dtype=float32) 0.18803493173916663
tf.Tensor([[ 0.5460856   0.99999976 -0.9535533  -0.9999996 ]], shape=(1, 4), dtype=float32) 0.23768013302485624
tf.Tensor([[-0.9994428   0.99999803 -0.99999976  0.9995004 ]], shape=(1, 4), dtype=float32) 0.31933263643582194
tf.Tensor([[ 0.99999976 -0.9840936  -0.99999976  0.64395046]], shape=(1, 4), dtype=float32) 0.3096375129818917
tf.Tensor([[ 0.9999995   0.7939516  -0.99999976  0.9673805 ]], shape=(1, 4), dtype=float32) 0.31409284579753877
tf.Tensor([[-0.9984421 -0.9933063 -0.9999759  0.9999962]], shape=(1, 4), dtype=float32) 0.30852211985985906
tf.Tensor([[-0.99999976 -0.9917878   0.18675382  0.9998551 ]], shape=(1, 4), dtype=float32) 0.42231888836622233
tf.Tensor([[-0.9999974   0.99999976 -0.96848065 -0.9280862 ]], shape=(1, 4), dtype=float32) 0.3101862555692641
tf.Tensor([[-0.9965702  -0.9997621   0.99999976 -0.99999356]], shape=(1, 4), dtype=float32) 0.0313026695996522

...

tf.Tensor([[-0.97927254 -0.9968779   0.9999997  -0.9999989 ]], shape=(1, 4), dtype=float32) 0.3107716727654329
tf.Tensor([[ 0.00423994 -0.988561    0.9999991  -0.99999976]], shape=(1, 4), dtype=float32) 0.33958663257456007
tf.Tensor([[ 0.6080908  -0.70417756  0.99999666 -0.99999976]], shape=(1, 4), dtype=float32) 0.2864261714021555
tf.Tensor([[ 0.6175905   0.5548554   0.99999106 -0.99999976]], shape=(1, 4), dtype=float32) 0.356832503447949
tf.Tensor([[-0.9780466   0.8618301   0.99999726 -0.99999976]], shape=(1, 4), dtype=float32) 0.16718722879386133
tf.Tensor([[-0.9667875   0.9986851   0.99998295 -0.99999976]], shape=(1, 4), dtype=float32) 0.08010142663621721
tf.Tensor([[-0.8908274   0.9913039   0.99998885 -0.99999976]], shape=(1, 4), dtype=float32) 0.14659078945714327
tf.Tensor([[-0.62443703  0.9846817   0.9999865  -0.99999976]], shape=(1, 4), dtype=float32) 0.19409931222599647
tf.Tensor([[ 0.53407514  0.99696714  0.99993294 -0.99999976]], shape=(1, 4), dtype=float32) 0.22766496747732

tf.Tensor([[-0.994767    0.5837336  -0.99999964  0.9999935 ]], shape=(1, 4), dtype=float32) 0.32638789331912993
tf.Tensor([[ 0.9997146  -0.96679735  0.99997854 -0.99999976]], shape=(1, 4), dtype=float32) 0.08677628584700915
tf.Tensor([[-0.9793034  -0.99994105  0.99999976 -0.9999933 ]], shape=(1, 4), dtype=float32) 0.10138816928863523
tf.Tensor([[ 0.9994802  -0.9051257   0.99993765 -0.99999976]], shape=(1, 4), dtype=float32) 0.043424983938572956
tf.Tensor([[ 0.99961615 -0.9094702  -0.16824894 -0.99888635]], shape=(1, 4), dtype=float32) 0.07812854113179536
tf.Tensor([[ 0.99010926  0.7384039  -0.99998534  0.86934394]], shape=(1, 4), dtype=float32) 0.08392231126624437
tf.Tensor([[-0.9990394   0.6366966  -0.99999946  0.9999948 ]], shape=(1, 4), dtype=float32) 0.275899528702138
tf.Tensor([[ 0.99999976 -0.7320455  -0.9999974  -0.9922762 ]], shape=(1, 4), dtype=float32) 0.2511868748267303
tf.Tensor([[-0.99999976 -0.99460614  0.6705137   0.99954015]], shape=(1, 4), dtype=float32) 0.277676032145

tf.Tensor([[-0.9989502  -0.9829434   0.99999976 -0.9999984 ]], shape=(1, 4), dtype=float32) 0.22286872075003297
tf.Tensor([[-0.97356594 -0.9551026   0.9999995  -0.9999997 ]], shape=(1, 4), dtype=float32) 0.2727502644658089
tf.Tensor([[ 0.8589777  -0.95670927  0.99999726 -0.99999976]], shape=(1, 4), dtype=float32) 0.21245648392039634
tf.Tensor([[ 0.9664821  -0.4932495   0.9999908  -0.99999976]], shape=(1, 4), dtype=float32) 0.15148494797945022
tf.Tensor([[ 0.999113    0.39883566  0.99991864 -0.99999976]], shape=(1, 4), dtype=float32) 0.08992118388414383
tf.Tensor([[-0.91733056 -0.9598545   0.9999994  -0.99999976]], shape=(1, 4), dtype=float32) 0.2357355135083199
tf.Tensor([[ 0.99859434  0.99999917  0.8183454  -0.99999976]], shape=(1, 4), dtype=float32) 0.18232658863067625
tf.Tensor([[-0.9987088   0.9989046   0.99999034 -0.99999976]], shape=(1, 4), dtype=float32) 0.25355558965605407
tf.Tensor([[-0.82288027  0.9997299   0.9999446  -0.99999976]], shape=(1, 4), dtype=float32) 0.302904292802

tf.Tensor([[ 0.9999942   0.11510964 -0.99999976  0.9970061 ]], shape=(1, 4), dtype=float32) 0.32011608026426935
tf.Tensor([[ 0.9997881  -0.94250154 -0.9998255   0.5868422 ]], shape=(1, 4), dtype=float32) 0.3146159373521804
tf.Tensor([[-0.5352002  -0.99992335  0.99999803 -0.99998933]], shape=(1, 4), dtype=float32) 0.24921133526164388
tf.Tensor([[-0.9938902  -0.9999952   0.99999976 -0.99996024]], shape=(1, 4), dtype=float32) 0.22549717152118684
tf.Tensor([[-0.9869118  -0.9999415   0.99999976 -0.9999927 ]], shape=(1, 4), dtype=float32) 0.29647134677570963
tf.Tensor([[-0.9938285  -0.9999699   0.99999976 -0.99998575]], shape=(1, 4), dtype=float32) 0.33681278824806216
tf.Tensor([[-0.9988743  -0.9997952   0.99999976 -0.9999842 ]], shape=(1, 4), dtype=float32) 0.32923728362721116
tf.Tensor([[-0.9895017   0.70415163  0.9999969  -0.99999976]], shape=(1, 4), dtype=float32) 0.29224045523005815
tf.Tensor([[ 0.74055684  0.9843473   0.99995655 -0.99999976]], shape=(1, 4), dtype=float32) 0.24109796053

tf.Tensor([[-0.9847449   0.6171532   0.9999982  -0.99999976]], shape=(1, 4), dtype=float32) 0.16296156754594948
tf.Tensor([[ 0.23454879  0.19737199  0.99999523 -0.99999976]], shape=(1, 4), dtype=float32) 0.18422051422297953
tf.Tensor([[ 0.963099   -0.2019292   0.9999889  -0.99999976]], shape=(1, 4), dtype=float32) 0.17274609228968624
tf.Tensor([[-0.69242084  0.99991846  0.999879   -0.99999976]], shape=(1, 4), dtype=float32) 0.2275151915724269
tf.Tensor([[ 0.9996844   0.9913083   0.9993593  -0.99999976]], shape=(1, 4), dtype=float32) 0.1433706633225083
tf.Tensor([[-0.30781096  0.9992962   0.9999411  -0.99999976]], shape=(1, 4), dtype=float32) 0.1724242638846049
tf.Tensor([[ 0.99908787  0.9941358   0.9993749  -0.99999976]], shape=(1, 4), dtype=float32) 0.08987444311380384
tf.Tensor([[ 0.99999976  0.87955767  0.56125367 -0.99999976]], shape=(1, 4), dtype=float32) 0.03858790411549898
tf.Tensor([[ 0.99999976 -0.64132696 -0.99928427 -0.99999106]], shape=(1, 4), dtype=float32) 0.0231395805279